In [1]:
import pandas as pd
import numpy as np

from datetime import date
Output_file = "_".join([str(date.today()), 'Country_AR.xlsx'])
Sheet_in_output_file = 'Export Worksheet'

In [2]:
data = pd.read_excel('2024_01_31.xlsx', sheet_name='Export Worksheet')

In [3]:
past_data = pd.read_excel('Country_AR_31.07.2023.xlsx', sheet_name='Export Worksheet')
country_merge_data = pd.read_excel('Country_AR_31.07.2023.xlsx', sheet_name='Overall AR')

In [ ]:
data_work = data

data_work['BU_country'] = np.NaN
data_work.loc[data_work['holding'] == 'EUROCHEM', 'BU_country'] = data_work.loc[data_work['holding'] == 'EUROCHEM', 'companyCode'].str[:2]
# Вычленение первых двух букв для страны в Еврохиме
data_work['BU_country_full'] = data_work.merge(country_merge_data, left_on='BU_country', right_on='EuroChem', how='left')['Unnamed: 0']
# Merge стран для Еврохима

data_work.loc[\
    ((data_work['holding'] == 'SUEK') & \
     (()data_work['orgName'].str.contains('"') | (data_work['orgName'].str.contains('ООО')))), \
        'BU_country'] = 'RU'
# Проставление RU для русских компаний СУЭК

SUEK_orgname_to_merge = data_work.loc[((data_work['holding'] == 'SUEK') & ~(data_work['orgName'].str.contains('"')) & ~(data_work['orgName'].str.contains('ООО'))), 'orgName'].unique().tolist()
# Список компаний СУЭК для merge
past_data_to_dict = past_data.loc[past_data.orgName.isin(SUEK_orgname_to_merge), ['orgName', 'BU country']].drop_duplicates()
SUEK_orgname_dict = {x:y for x,y in zip(past_data_to_dict.orgName, past_data_to_dict['BU country'])}
# Спиосок компаний СУЭК для переименованя из прошлых данных

data_work.loc[((data_work.BU_country.isna()) & (data_work.holding != 'EXTERNAL')), 'BU_country_full'] =\
      data_work.loc[((data_work.BU_country.isna()) & (data_work.holding != 'EXTERNAL')), 'orgName']
data_work.BU_country_full =  data_work.BU_country_full.replace(SUEK_orgname_dict)
# Переименование по СУЭК диск и запись в data_work

RU_dict = {'RU': 'Russia'}
data_work['BU_country_full'] = data_work.BU_country_full.replace(RU_dict)

In [17]:
data_work['BU_country_full'].isna().value_counts()

BU_country_full
False    399825
Name: count, dtype: int64

In [ ]:
data_work['BU_country=CP_country'] = data_work['counterpartyCountryName'] == data_work['BU_country_full']

filter_accountType_list = ['Other receivables', 'Provisions', 'Trade receivables', 'Advances to suppliers']

SUEK_data = data_work.query('holding == "SUEK"')
SUEK_data_Ex = SUEK_data.loc[((SUEK_data['BU_country=CP_country'] == False) & (SUEK_data.internalExternalEN == "External") & (SUEK_data['accountType'].isin(filter_accountType_list)))]
SUEK_data_In = SUEK_data.loc[((SUEK_data['BU_country=CP_country'] == False) & (SUEK_data.internalExternalEN == "Internal") & (SUEK_data['accountType'].isin(filter_accountType_list)))]

ECH_data = data_work.query('holding == "EUROCHEM"')
ECH_data_Ex = ECH_data.loc[((ECH_data['BU_country=CP_country'] == False) & (ECH_data.internalExternalEN == "External") & (ECH_data['accountType'].isin(filter_accountType_list)))]
ECH_data_In = ECH_data.loc[((ECH_data['BU_country=CP_country'] == False) & (ECH_data.internalExternalEN == "Internal") & (ECH_data['accountType'].isin(filter_accountType_list)))]

BU_country=CP_country
True     382281
False     17544
Name: count, dtype: int64

In [22]:
# data_work.to_excel(Output_file, sheet_name=Sheet_in_output_file, index=False)

In [ ]:
SUEK_data_Ex_pivot = SUEK_data_Ex.pivot_table(values='presentAmountUSD',\
                        index = 'counterpartyCountryName',\
                        aggfunc=sum).reset_index()
SUEK_data_In_pivot = SUEK_data_In.pivot_table(values='presentAmountUSD',\
                        index = 'counterpartyCountryName',\
                        aggfunc=sum).reset_index()
ECH_data_Ex_pivot = ECH_data_Ex.pivot_table(values='presentAmountUSD',\
                        index = 'counterpartyCountryName',\
                        aggfunc=sum).reset_index()
ECH_data_In_pivot = ECH_data_In.pivot_table(values='presentAmountUSD',\
                        index = 'counterpartyCountryName',\
                        aggfunc=sum).reset_index()

SUEK_data_Ex_pivot.to_excel(Output_file,sheet_name='SUEK_Ex', index=False)
with pd.ExcelWriter(Output_file, engine='openpyxl', mode='a') as writer:
    # SUEK_data_Ex_pivot.to_excel(writer,sheet_name='SUEK_Ex', index=False)
    SUEK_data_In_pivot.to_excel(writer,sheet_name='SUEK_In', index=False)
    ECH_data_Ex_pivot.to_excel(writer,sheet_name='ECH_Ex', index=False)
    ECH_data_In_pivot.to_excel(writer,sheet_name='ECH_In', index=False)
    # data_work.to_excel(writer, sheet_name=Sheet_in_output_file, index=False)

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_15532\4193317742.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  SUEK_data_Ex_pivot = SUEK_data_Ex.pivot_table(values='presentAmountUSD',\
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_15532\4193317742.py:4: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  SUEK_data_In_pivot = SUEK_data_In.pivot_table(values='presentAmountUSD',\
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_15532\4193317742.py:7: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable wil

In [24]:
ECH_data_In_pivot.head(2)

,counterpartyCountryName,presentAmountUSD
0,Argentina,3.514170e+07
1,Belgium,1.353413e+08


In [25]:
check = data_work.query('holding == "EUROCHEM" & internalExternalEN =="Internal"')
check = check.loc[check['BU_country=CP_country'] == False]
check = check.loc[check['accountType'].isin(filter_accountType_list)]
check.loc[check['counterpartyCountryName'] == 'Argentina'].presentAmountUSD.sum()

35141701.51

In [27]:
data_work.columns

Index(['rowType', 'reportType', 'reportDate', 'orgName', 'companyCode',
       'codeBUsap', 'counterpartyNumber', 'partyInternational',
       'internalExternalRU', 'internalExternalEN', 'partnerCategoryEN',
       'partnerCategoryRU', 'groupRU', 'groupEN', 'counterpartyType',
       'docTypeRU', 'countryReg', 'counterpartyCountryName',
       'functionalCurrency', 'amountDocFunctional', 'amountFunctional',
       'presentAmountUSD', 'reservAmount', 'debtHoldingCurrency',
       'histAmountHoldCur', 'documentDate', 'dueDate', 'overdueDays',
       'drDescription', 'debtDescription', 'itemDescription',
       'termDescription', 'glAccount', 'accountType', 'glBalancing', 'bscCode',
       'bsxValue', 'sourceSystem', 'creditId', 'ratingCam', 'holding',
       'BU_country', 'BU_country_full', 'BU_country=CP_country'],
      dtype='object')

In [38]:
check2 = data_work[['holding', 'orgName', 'companyCode', 'BU_country', 'BU_country_full','counterpartyCountryName', 'BU_country=CP_country', 'internalExternalEN', 'presentAmountUSD', 'accountType']]
check2.query("BU_country_full != 'Russia' & holding == 'SUEK'").orgName.unique()
check2.query("holding == 'SUEK'").loc[~((check2.orgName.str.contains('ООО')) | (check2.orgName.str.contains('"'))), ['orgName', 'BU_country_full']].drop_duplicates()

,orgName,BU_country_full
246984,Black Sand Commodities,UAE
255482,SUEK AG (Swiss),Switzerland
255484,SUEK Logistic Moscow,Russia
255579,SST Shanghai,China
255630,SST Harbin,China
256480,SUEK Polska,Poland
259388,SUEK Logistic (Swiss),Switzerland
269315,SUEK Securities D.A.C.,Russia
283686,SUEK Korea Ltd,South Korea
295956,FERCO - Cyprus,Cyprus
